In [1]:
import pymongo
print(pymongo.__version__)

4.7.3


### Connect

In [6]:
#server
client = pymongo.MongoClient("mongodb://localhost:27017/")
#db
mydb = client["Engines"]
#colections
carinfo = mydb.carInformation
engineinfo = mydb.engineInformation

In [8]:
for record in engineinfo.find():
    print(record)

{'_id': ObjectId('667613fdfc5299048e577843'), 'engine_name': 'LS3', 'manufacture': 'GM', 'capacity': '6.2L', 'horse_power': 425}
{'_id': ObjectId('66761554fc5299048e577844'), 'engine_name': '2JZ', 'manufacture': 'Toyota', 'capacity': '3.0L', 'horse_power': 350}
{'_id': ObjectId('66761554fc5299048e577845'), 'engine_name': 'Cayote', 'manufacture': 'Ford', 'capacity': '5.0L', 'horse_power': 475}
{'_id': ObjectId('66761554fc5299048e577846'), 'engine_name': 'Godzilla', 'manufacture': 'Ford', 'capacity': '7.0L', 'horse_power': 525}


In [23]:
for record in engineinfo.find():
    engine_name = record['engine_name']
    capacity = record['capacity']
    capacity = float(capacity[:-1])
    engineinfo.update_one(
        {'engine_name':engine_name},
        {'$set':{'capacity':capacity}}
    )
for record in engineinfo.find():
    print(record)

{'_id': ObjectId('667613fdfc5299048e577843'), 'engine_name': 'LS3', 'manufacture': 'GM', 'capacity': 6.2, 'horse_power': 425}
{'_id': ObjectId('66761554fc5299048e577844'), 'engine_name': '2JZ', 'manufacture': 'Toyota', 'capacity': 3.0, 'horse_power': 350}
{'_id': ObjectId('66761554fc5299048e577845'), 'engine_name': 'Cayote', 'manufacture': 'Ford', 'capacity': 5.0, 'horse_power': 475}
{'_id': ObjectId('66761554fc5299048e577846'), 'engine_name': 'Godzilla', 'manufacture': 'Ford', 'capacity': 7.0, 'horse_power': 525}


In [24]:
new_data = [
    {
    "engine_name":"K20",
    "manufacture":"Honda",
    "capacity":2.0,
    "horse_power":165
    },
    {
    "engine_name":"B16B",
    "manufacture":"Honda",
    "capacity":2.0,
    "horse_power":175
    },
    {
    "engine_name":"Cummins",
    "manufacture":"Dodge",
    "capacity":7.0,
    "horse_power":465
    },
    {
    "engine_name":"PowerStroke",
    "manufacture":"Ford",
    "capacity":7.0,
    "horse_power":550
    }
]
engineinfo.insert_many(new_data)
for record in engineinfo.find():
    print(record)

{'_id': ObjectId('667613fdfc5299048e577843'), 'engine_name': 'LS3', 'manufacture': 'GM', 'capacity': 6.2, 'horse_power': 425}
{'_id': ObjectId('66761554fc5299048e577844'), 'engine_name': '2JZ', 'manufacture': 'Toyota', 'capacity': 3.0, 'horse_power': 350}
{'_id': ObjectId('66761554fc5299048e577845'), 'engine_name': 'Cayote', 'manufacture': 'Ford', 'capacity': 5.0, 'horse_power': 475}
{'_id': ObjectId('66761554fc5299048e577846'), 'engine_name': 'Godzilla', 'manufacture': 'Ford', 'capacity': 7.0, 'horse_power': 525}
{'_id': ObjectId('6677893e0a4b59c8d1b0a8de'), 'engine_name': 'K20', 'manufacture': 'Honda', 'capacity': 2.0, 'horse_power': 165}
{'_id': ObjectId('6677893e0a4b59c8d1b0a8df'), 'engine_name': 'B16B', 'manufacture': 'Honda', 'capacity': 2.0, 'horse_power': 175}
{'_id': ObjectId('6677893e0a4b59c8d1b0a8e0'), 'engine_name': 'Cummins', 'manufacture': 'Dodge', 'capacity': 7.0, 'horse_power': 465}
{'_id': ObjectId('6677893e0a4b59c8d1b0a8e1'), 'engine_name': 'PowerStroke', 'manufacture

### Aggregate Functions      
* Aggreation functions are used to process and transform data within a collection to produce computed results.

**Key Concepts of MongoDB Aggregation Framework**           
1. Aggregation Pipeline:
* The agregation framework uses a pipleine approach, where multiple stages are chained togeather to process documents.
* Each stage perfrms an operation on the input document and passes the results to the next stage.       
2. Stages:
* Each stage in the pipline represents an operation (e.g filtering, grouping, projecting)
* Common stages include **match, group, sort, project, limit, skip, unwind, lookup** 

In [33]:
#$match - filter
results = engineinfo.aggregate([
                {'$match':{'capacity':{'$gt':5}}}
                                ])
for result in results:
    print(result)

{'_id': ObjectId('667613fdfc5299048e577843'), 'engine_name': 'LS3', 'manufacture': 'GM', 'capacity': 6.2, 'horse_power': 425}
{'_id': ObjectId('66761554fc5299048e577846'), 'engine_name': 'Godzilla', 'manufacture': 'Ford', 'capacity': 7.0, 'horse_power': 525}
{'_id': ObjectId('6677893e0a4b59c8d1b0a8e0'), 'engine_name': 'Cummins', 'manufacture': 'Dodge', 'capacity': 7.0, 'horse_power': 465}
{'_id': ObjectId('6677893e0a4b59c8d1b0a8e1'), 'engine_name': 'PowerStroke', 'manufacture': 'Ford', 'capacity': 7.0, 'horse_power': 550}


In [43]:
#$group - groups documents
results = engineinfo.aggregate([
    {'$group':
             {'_id':'$manufacture', 
              'total_power':{'$sum':'$horse_power'}
             }
    }
])
for result in results:
    print(result)

{'_id': 'Ford', 'total_power': 1550}
{'_id': 'GM', 'total_power': 425}
{'_id': 'Honda', 'total_power': 340}
{'_id': 'Dodge', 'total_power': 465}
{'_id': 'Toyota', 'total_power': 350}


In [62]:
#$project - reshapes docs by including, excluding or adding new fields
#include
results = engineinfo.aggregate([
    {'$match':{'manufacture':'Ford'}},      #filter
    {'$limit':3},                           #limit
    {'$skip':1},                            #skip     
    {'$sort':{'engine_name':-1}},           #sort
    {
        '$project':{
            'engine':'$engine_name',
            'usable_power': {'$multiply':['$horse_power',0.75]},
            'manufacture': 1
        }
    }
     ])
for result in results:
    print(result)

{'_id': ObjectId('6677893e0a4b59c8d1b0a8e1'), 'manufacture': 'Ford', 'engine': 'PowerStroke', 'usable_power': 412.5}
{'_id': ObjectId('66761554fc5299048e577846'), 'manufacture': 'Ford', 'engine': 'Godzilla', 'usable_power': 393.75}


In [63]:
#$unwind - if certain filed have a list of values then one entry will create n entries for each value